In [1]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler()
X_test_scaled = scaler.fit_transform(X_test)


scaler = MinMaxScaler()
X_test_scaled = scaler.fit_transform(X_test)

# NaN이 있는 행 제거
X_test_clean = X_test_scaled[~np.isnan(X_test_scaled).any(axis=1)]

# 예측
prediction = model.predict(X_test_clean)

NameError: name 'X_test' is not defined

In [ ]:
import numpy as np

# NaN이 있는 행 제거
X_test_clean = X_test_scaled[~np.isnan(X_test_scaled).any(axis=1)]

# 예측
prediction = model.predict(X_test_clean)

In [ ]:
prediction

inference에 필요한 features

In [ ]:
import ee

try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()

# location setting
lat, lon = 28.565038182965658, -81.15889424232911
point = ee.Geometry.Point(lon, lat).buffer(5000)

In [ ]:
import pandas as pd

def ee_fc_to_pandas_safe(fc, keys):
    arrays = {k: fc.aggregate_array(k).getInfo() for k in keys}
    min_len = min(len(v) for v in arrays.values())
    arrays = {k: v[:min_len] for k, v in arrays.items()}
    return pd.DataFrame(arrays)
def extract(img):
    mean = img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=250,
        bestEffort=True
    )
    return ee.Feature(None, {
        "EVI": mean.get("EVI"),
        "date": ee.Date(img.get("system:time_start")).format("YYYY-MM-dd")
    })
def get_modis_evi(point, start_date, end_date):
    """MODIS EVI 데이터를 GEE에서 불러와 pandas DataFrame으로 변환"""
    modis = (
        ee.ImageCollection("MODIS/006/MOD13Q1")
        .filterDate(start_date, end_date)
        .select("EVI")
        .map(lambda img: img.multiply(0.0001).copyProperties(img, ["system:time_start"]))
    )

    def extract(img):
        mean = img.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=250,
            bestEffort=True
        )
        return ee.Feature(None, {
            "EVI": mean.get("EVI"),
            "date": ee.Date(img.get("system:time_start")).format("YYYY-MM-dd")
        })

    evi_fc = modis.map(extract)
    evi_df = ee_fc_to_pandas_safe(evi_fc, ["EVI", "date"])

    # 후처리
    evi_df["date"] = pd.to_datetime(evi_df["date"])
    evi_df["DOY"] = evi_df["date"].dt.dayofyear
    evi_df = evi_df.sort_values("date").reset_index(drop=True)
    return evi_df

prediction

In [ ]:
import joblib

model = joblib.load(r'C:\sehwi\NASAHackerton\data\output\saved_models\ridge_model.joblib')
model

In [ ]:
import numpy as np

# NaN이 있는 행 제거
X_test_clean = X_test_scaled[~np.isnan(X_test_scaled).any(axis=1)]

# 예측
prediction = model.predict(X_test_clean)

In [ ]:
X_test = filtered_df.values.tolist()
X_test
X_test = filtered_df[['tmax', 'tmin', 'dayl', 'prcp', 'vp']]